In [ ]:
from vantage6.client import Client
import yaml

In [ ]:
client = Client("https://v6.server.net", 443, "/api")
client.authenticate("root", "root")
client.setup_encryption(None)

In [ ]:
# Check what orgs we already have
all_orgs = client.organization.list()

for org in all_orgs:
    print(f'{org["id"]}: {org["name"]}')

In [ ]:
# Just to hold any orgs we create
new_orgs = []

In [ ]:
# Add orgs from a yaml file (YAML needs at the very least a domain and name)
with open('orgs.yaml') as f:
    org_inputs = yaml.safe_load(f)

for org_input in org_inputs:
    print(f'Parsing org {org_input["name"]}')
    if org_input['domain'].lower() in [org['domain'].lower() for org in all_orgs if org['domain']]:
        print('Already have this org on the server, skipping')

    new_org = client.organization.create(**org_input)
    if new_org:
        print(f'New org has id {new_org["id"]}')
        new_orgs.append(new_org)

print(new_orgs)

In [ ]:
# Manually add an org
new_org = client.organization.create(
    name="",
    address1="",
    address2="",
    zipcode="",
    country="",
    domain=""
)

print(new_org)
new_orgs.append(new_org)

In [ ]:
# If you want to add more rules, listing them here:
all_rules = client.rule.list()
for rule in all_rules:
    print(f'{rule["id"]} - {rule["operation"]} {rule["name"]} - {rule["scope"]}')

In [ ]:
# Specify roles for org admins
all_rules = client.rule.list()

# Can do anything at ORGANIZATION level
rules = [rule['id'] for rule in all_rules if rule['scope'] == 'ORGANIZATION']

# Remove some rules
rules = [
    rule for rule in rules if rule not in [
        8, # Delete node
    ]
]

In [ ]:
# Add org admin user for each (we do need one for each to upload private keys, even if they don't intend to use it)
# Uses orgname-admin@medicaldataworks.nl as email as at this point we don't want to have to put in contact details
# For passwords:
import secrets
import string
import re
alphabet = string.ascii_letters + string.digits

new_users = []

for org in new_orgs:
    name = re.sub(r'[^0-9a-zA-Z]+', '-', org["name"].lower())
    username = f'{name}-admin'
    password = ''.join(secrets.choice(alphabet) for i in range(20))

    new_user = client.user.create(
        username=username,
        firstname='',
        lastname='',
        password=password,
        email=f'{name}-admin@medicaldataworks.nl',
        organization=org['id'],
        rules = rules
    )

    if new_user:
        # Check that this has actually been assigned to the correct org
        if new_user['organization']['id'] != org['id']:
            new_user = client.user.update(
                id_ = new_user['id'],
                organization=org['id']
            )
            
        print(f'{username} - {password}')
        new_users.append(new_user)

print(new_users)